In [187]:
import os
from nltk.tokenize import word_tokenize

dir = 'russian_corpus'
filepaths = []

for i in os.listdir(dir):
    filepaths.append(os.path.join(dir, i))  

corpus = []
tokens = []

for filepath in filepaths:
    with open(filepath, encoding='utf-8') as f:
        text = f.read()
        text.replace('.', '')
        text = [x for x in word_tokenize(text) if x.isalpha()]
        if text:
            tokens.extend(text)
            text = [x for x in [text[0]] if len(x) > 3] + [x for x in text[1:] if not x.istitle() and len(x) > 3]
            text = ' '.join(text)
            corpus.append(text)

print(f'Количество словоформ: {len(tokens)}')
print(f'Количество токенов: {len(set(tokens))}')
print(f'Количество документов в корпусе: {len(corpus)}')

with open('ru_stop_words.txt', encoding='utf-8') as f:
    STOPWORDS = f.read().split()
    print(STOPWORDS)

Количество словоформ: 232763
Количество токенов: 31861
Количество документов в корпусе: 23016
['a', 'about', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', 'could', 'do', 'for', 'from', 'has', 'have', 'i', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'not', 'of', 'on', 'one', 'or', 'so', 'that', 'the', 'them', 'there', 'they', 'this', 'to', 'was', 'we', 'what', 'which', 'will', 'with', 'would', 'you', 'а', 'будем', 'будет', 'будете', 'будешь', 'буду', 'будут', 'будучи', 'будь', 'будьте', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вами', 'вас', 'весь', 'во', 'вот', 'все', 'всё', 'всего', 'всей', 'всем', 'всём', 'всеми', 'всему', 'всех', 'всею', 'всея', 'всю', 'вся', 'вы', 'да', 'для', 'до', 'его', 'едим', 'едят', 'ее', 'её', 'ей', 'ел', 'ела', 'ем', 'ему', 'емъ', 'если', 'ест', 'есть', 'ешь', 'еще', 'ещё', 'ею', 'же', 'за', 'и', 'из', 'или', 'им', 'ими', 'имъ', 'их', 'к', 'как', 'кем', 'ко', 'когда', 'кого', 'ком', 'кому', 'комья', 'кот

In [188]:
import random

for i in range(100):
    print(random.choice(tokens))

подмигивая
сказала
это
между
которые
за
Всё
говорим
мне
Это
о
он
милая
кузнецы
предсказала
котором
вопроса
генерал
У
о
где
большая
а
они
ибо
его
Ивановна
насмешливою
сказал
это
ждать
было
бы
не
Не
подействовали
нам
конкурентов
Бог
мне
говорить
слез
оставишь
шопотом
я
и
холодность
что
сказала
слушает
знал
изо
на
Всё
Кити
князь
писал
на
рада
qui
много
и
адъютантом
вместе
А
Козловского
деньги
сказал
внаймы
выгоняет
надулось
сами
ветра
доживает
тело
отвечать
правда
Николенька
возможно
к
познакомила
этого
и
Я
то
disons
бабушка
толпилось
в
Он
вас
но
этом
нет
как
знаю
всем
больше
А
играет


In [194]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

no_features = 31000
no_topics = 10

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.06,
    # min_df=3, 
    stop_words=STOPWORDS, 
    max_features=no_features
)
data_tfidf_vectorized = tfidf_vectorizer.fit_transform(corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


nmf = NMF(n_components=no_topics, 
          random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd'
          )
nmf_tfidf_model = nmf.fit(data_tfidf_vectorized)
print(nmf_tfidf_model.transform(data_tfidf_vectorized).shape)

(23016, 10)


In [ ]:
def print_topics(model, feature_names, top_n=10):
    for topic_idx, topic in enumerate(model.components_):
            print("Topic %d:" % (topic_idx))
            print(" ".join([feature_names[i] for i in topic.argsort()[:-top_n - 1:-1]]))

In [195]:
print_topics(nmf_tfidf_model, tfidf_feature_names)


Topic 0:
хорошо слишком лучше жили правда прекрасно помнишь садись отвечал здесь
Topic 1:
ничего понимаю нужно отвечал больше хочу кроме никогда сделать особенного
Topic 2:
отчего думаете можно ехать знаешь говорить слишком хотите потрудиться говорите
Topic 3:
князь старый отвечал перебил продолжал повторил вдруг думаете тихо князю
Topic 4:
очень рада видеть интересно приятно человек приехал весело люблю больна
Topic 5:
теперь дело надо куда говорить больше время ехать здесь ведь
Topic 6:
знаю право сказать думаю говорю княжна люблю давно говорить ведь
Topic 7:
делать нечего скажи надо народом вашему скажу трудно спать аккуратно
Topic 8:
зачем здесь говоришь едете говорите пришла поедете служите умерла взял
Topic 9:
улыбаясь весело прибавил отвечал думаю глядя нежно отвечала говорили лучше
